<a href="https://colab.research.google.com/github/aveerdram/JSON/blob/main/%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%B7%D0%B0%D1%91%D0%BC%D1%89%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перед оформлением заявки на получение кредита потенциальный заемщик
проходит ряд автоматических стоп-проверок. Если хотя бы одну из этих
проверок клиент не прошел, то банк вправе отказать ему в выдаче кредита. Ниже
представлены три проверки, которые требуется реализовать:

**Проверка 1. Минимальный возраст.**

Если клиент моложе 20 лет – отказ

**Проверка 2. Проверка действительности паспорта.**

Если возраст клиента более 20 лет и дата выдачи паспорта ранее, чем дата
достижения им возраста 20 лет, либо если возраст клиента более 45 лет и дата
выдачи паспорта ранее, чем дата достижения им возраста 45 лет - отказ.


**Проверка 3. Проверка кредитной истории.**

Наличие в кредитной истории в Банке хотя бы одного из условий приводит к
отказу.
Если тип кредита не «Кредитная карта», то проверяется:
1. Имеется непогашенная просроченная задолженность.
2. Возникала просроченная задолженность протяженностью более 60 дней.
3. Есть больше двух кредитов с просроченной задолженностью
протяженностью более 15 дней.

Если тип кредита «Кредитная карта», то проверяется:
1. Имеется непогашенная просроченная задолженность.
2. Возникала просроченная задолженность протяженностью более 30 дней.

# Тестовый файл

In [ ]:
with open('test.json', 'w') as in_file:
  in_file.write(
'''{
  "firstName": "Иван",
  "middleName": "Иванович",
  "lastName": "Иванов",
  "birthDate": "1969-12-31T21:00:00.000Z",
  "citizenship": "РФ",
  "passport": {
    "series": "12 34",
    "number": "123456",
    "issuedAt": "2023-03-11T21:00:00.000Z",
    "issuer": "УФМС",
    "issuerСode": "123-456"
  },
  "creditHistory": [
    {
      "type": "Кредит наличными",
      "currency": "RUB",
      "issuedAt": "2003-02-27T21:00:00.000Z",
      "rate": 0.13,
      "loanSum": 100000,
      "term": 12,
      "repaidAt": "2004-02-27T21:00:00.000Z",
      "currentOverdueDebt": 0,
      "numberOfDaysOnOverdue": 0,
      "remainingDebt": 0,
      "creditId": "25e8a350-fbbc-11ee-a951-0242ac120002"
    },
    {
      "type": "Кредитная карта",
      "currency": "RUB",
      "issuedAt": "2009-03-27T21:00:00.000Z",
      "rate": 0.24,
      "loanSum": 30000,
      "term": 3,
      "repaidAt": "2009-06-29T20:00:00.000Z",
      "currentOverdueDebt": 0,
      "numberOfDaysOnOverdue": 2,
      "remainingDebt": 0,
      "creditId": "81fb1ff6-fbbc-11ee-a951-0242ac120002"
    },
    {
      "type": "Кредит наличными",
      "currency": "RUB",
      "issuedAt": "2009-02-27T21:00:00.000Z",
      "rate": 0.09,
      "loanSum": 200000,
      "term": 24,
      "repaidAt": "2011-03-02T21:00:00.000Z",
      "currentOverdueDebt": 0,
      "numberOfDaysOnOverdue": 14,
      "remainingDebt": 0,
      "creditId": "c384eea2-fbbc-11ee-a951-0242ac120002"
    },
    {
      "type": "Кредитная наличными",
      "currency": "RUB",
      "issuedAt": "2024-05-15T21:00:00.000Z",
      "rate": 0.13,
      "loanSum": 200000,
      "term": 36,
      "repaidAt": null,
      "currentOverdueDebt": 10379,
      "numberOfDaysOnOverdue": 15,
      "remainingDebt": 110000,
      "creditId": "ebeddfde-fbbc-11ee-a951-0242ac120002"
    }
  ]
}
''')

# **Функция**

In [ ]:
import json
from datetime import datetime

def check_client(json_data):
  """
  Функция для проверки клиента на соответствие стоп-критериям.

  Args:
      json_data: JSON-данные о клиенте и его кредитной истории.

  Returns:
      True, если клиент прошел все проверки, False в противном случае.
  """

  def str_to_datetime(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")

  with open(json_data, 'r') as fcc_file:
    client_data = json.load(fcc_file)

    # Проверка 1: Минимальный возраст
    age = datetime.today().year - int(client_data.get('birthDate')[:4])
    if age < 20:
      return False

    # Проверка 2: Проверка действительности паспорта

    passport = client_data.get('passport')
    issue_date = str_to_datetime(passport.get('issuedAt'))
    birth_date = str_to_datetime(client_data.get('birthDate'))

    if age > 45:
      date45 = birth_date.replace(year=birth_date.year + 45)
      d = date45 - issue_date
      if d.days > 0:
        return False
    elif age > 20:
      date20 = birth_date.replace(year=birth_date.year + 20)
      d = date20 - issue_date
      if d.days > 0:
        return False
    else:
        return False

    # Проверка 3: Проверка кредитной истории
    credit_list = client_data.get('creditHistory')
    flag = -2

    for credit in credit_list:
      if credit.get('type') == 'Кредитная карта':
        if credit.get('currentOverdueDebt') != 0 or credit.get('numberOfDaysOnOverdue') > 30:
          return False
    else:
      if credit.get('numberOfDaysOnOverdue') > 15:
        flag += 1
      if credit.get('currentOverdueDebt') != 0 or credit.get('numberOfDaysOnOverdue') > 60 or flag >= 0:
        return False

    return True

In [ ]:
check_client('test.json')

False